# Tutaj rozpoczyna się proces rozpoznawania działań.

Wczytanie modelu

In [10]:
import numpy as np 
import keras
import tensorflow as tf
import PIL
import cv2 # Biblioteka pozwalająca na odczytywanie poszczególnych partii obrazu.

my_model = keras.saving.load_model('equation_solver_v5.keras', custom_objects=None, compile=True, safe_mode=True)

my_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 45, 45, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 45, 45, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 22, 22, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 22, 22, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 11, 11, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 688,931 (2.63 MB)

 Trainable params: 229,643 (897.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 459,288 (1.75 MB)

# Załadowanie nazw klas i parametrów

In [11]:
IMG_WIDTH = 45
IMG_HEIGHT = 45
class_names = ['+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# Przykładowe użycie modelu do rozpoznania liczby

In [12]:
img = PIL.Image.open('examples/four.png').convert('L')
img = img.resize((IMG_WIDTH, IMG_HEIGHT))
img_array = keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = my_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
This image most likely belongs to 4 with a 90.38 percent confidence.


:)

# Wyświetlenie obrazu działania matematycznego

In [13]:
image = cv2.imread('examples/2137plus420.png', cv2.IMREAD_GRAYSCALE)

cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Przekonwertowanie obrazu na binarny

In [14]:
binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 4)

cv2.imshow('Image', binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Znalezienie poszczególnych liczb i znaków na obrazie

In [15]:
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])
contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(contour_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('Image', contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Filtrowanie ewentualnego szumu z obrazu

In [16]:
filtered_rectangles = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    new_rect = (x, y, x + w, y + h)

    overlap = False
    for rect in filtered_rectangles:
        if (new_rect[0] < rect[2] and new_rect[2] > rect[0] and
            new_rect[1] < rect[3] and new_rect[3] > rect[1]):
            overlap = True
            break
    
    if not overlap:
        filtered_rectangles.append(new_rect)

for rect in filtered_rectangles:
    cv2.rectangle(contour_image, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
    
cv2.imshow('Image', contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Skalowanie wyciętych liczb do 45x45

In [17]:
resized_images = []
target_size = (45, 45)

for rect in filtered_rectangles:
    x1, y1, x2, y2 = rect
    roi = image[y1:y2, x1:x2]
    resized_roi = cv2.resize(roi, target_size)
    resized_images.append(resized_roi)
    
for i, resized_image in enumerate(resized_images):
    cv2.imshow(f'Image {i}', resized_image)
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [18]:
predicted_symbols = []

for resized_image in resized_images:
    resized_image_normalized = np.expand_dims(resized_image, axis=-1)
    resized_image_normalized = np.expand_dims(resized_image_normalized, axis=0)
    prediction = my_model.predict(resized_image_normalized)
    predicted_class_index = np.argmax(prediction)
    predicted_symbol = class_names[predicted_class_index]
    predicted_symbols.append(predicted_symbol)

predicted_string = ''.join(predicted_symbols)
print(f"Predicted string: {predicted_string}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted string: 2137+420
